In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!git clone https://github.com/wl-research/nubia.git
import os
os.chdir('nubia')

fatal: destination path 'nubia' already exists and is not an empty directory.


In [ ]:
!pip install -r requirements.txt

In [ ]:
import pandas as pd
import numpy as np
import string
from string import digits
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
from nubia import Nubia
import datetime

Using TensorFlow backend.


In [ ]:
nubia = Nubia()

loading archive file pretrained/roBERTa_STS
| [input] dictionary: 50265 types
loading archive file pretrained/roBERTa_MNLI
| dictionary: 50264 types


In [ ]:
lines= pd.read_table('/content/drive/My Drive/rus.txt', names=['eng','rus', '1'])

In [ ]:
lines = lines.drop(['1'], axis=1)
lines = lines[:38696]
lines = shuffle(lines)

In [ ]:
lines.shape

(38696, 2)

In [ ]:
# Lowercase all characters
lines.eng=lines.eng.apply(lambda x: x.lower())
lines.rus=lines.rus.apply(lambda x: x.lower())

# Remove quotes
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x))
lines.rus=lines.rus.apply(lambda x: re.sub("'", '', x))

exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.rus=lines.rus.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines.eng = lines.eng.apply(lambda x: x.translate(remove_digits))
lines.rus = lines.rus.apply(lambda x: x.translate(remove_digits))

# Remove extra spaces
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.rus=lines.rus.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.rus=lines.rus.apply(lambda x: re.sub(" +", " ", x))

# Add start and end tokens to target sequences
lines.eng = lines.eng.apply(lambda x : 'START_ '+ x + ' _END')

In [ ]:
lines.sample(10)

,eng,rus
36378,START_ do you bind books _END,ты занимаешься переплетением книг
33190,START_ tom isnt around _END,тома поблизости нет
5500,START_ are you busy _END,ты занят
13659,START_ do you do yoga _END,вы занимаетесь йогой
18338,START_ why am i alive _END,почему я жив
24526,START_ tom looks drunk _END,том выглядит пьяным
7302,START_ math is hard _END,математика сложна
10133,START_ i was running _END,я бежала
14329,START_ hows your dad _END,как твой папенька
38477,START_ i heard them sing _END,я слышал как они поют


In [ ]:
# Vocabulary of Russian
all_rus_words=set()
for rus in lines.rus:
    for word in rus.split():
        if word not in all_rus_words:
            all_rus_words.add(word)

# Vocabulary of English 
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

In [ ]:
# Max Length of source sequence
lenght_list=[]
for l in lines.rus:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
max_length_src

10

In [ ]:
# Max Length of target sequence
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
max_length_tar

8

In [ ]:
input_words = sorted(list(all_rus_words))
target_words = sorted(list(all_eng_words))
num_encoder_tokens = len(all_rus_words)
num_decoder_tokens = len(all_eng_words)
num_encoder_tokens, num_decoder_tokens

(11582, 4207)

In [ ]:
num_decoder_tokens += 1 # For zero padding
num_encoder_tokens +=1
num_decoder_tokens

4208

In [ ]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [ ]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [ ]:
lines.head(10)

,eng,rus
23729,START_ they told me so _END,они мне так сказали
25497,START_ whats his name _END,как его имя
21879,START_ im glad i came _END,я рад что пришёл
13106,START_ youre biased _END,ты предвзята
12043,START_ tom is a baby _END,том младенец
35266,START_ you seem content _END,ты вроде довольна
4300,START_ is it cheap _END,оно дешёвое
32944,START_ tom is a fat man _END,том дородный
7365,START_ now try this _END,теперь попробуй это
21481,START_ i was very cold _END,мне было очень холодно


In [ ]:
#Split the data into train and test
X, y = lines.rus, lines.eng
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((34826,), (3870,))

In [ ]:
#Define a function which generates the data for train and test in batches

def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [ ]:
#Define the encoder-decoder architecture with LSTM
latent_dim = 50

In [ ]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)

encoder_states = [state_h, state_c]

In [ ]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 32
epochs = 100

In [ ]:
model.load_weights('/content/drive/My Drive/Colab Notebooks/weights.h5')

In [ ]:
print('Beginning Training!') 
current_time = datetime.datetime.now() 

model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = 200,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

print("Training took time ", datetime.datetime.now() - current_time)

Beginning Training!


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/100
200/200 [==============================] - 18s 89ms/step - loss: 0.9586 - acc: 0.7633 - val_loss: 1.0436 - val_acc: 0.7636
Epoch 2/100
200/200 [==============================] - 17s 84ms/step - loss: 0.9782 - acc: 0.7569 - val_loss: 0.9939 - val_acc: 0.7605
Epoch 3/100
200/200 [==============================] - 17s 83ms/step - loss: 0.9431 - acc: 0.7634 - val_loss: 0.8199 - val_acc: 0.7597
Epoch 4/100
200/200 [==============================] - 17s 83ms/step - loss: 0.9697 - acc: 0.7558 - val_loss: 1.0353 - val_acc: 0.7605
Epoch 5/100
200/200 [==============================] - 17s 84ms/step - loss: 0.9821 - acc: 0.7596 - val_loss: 0.9861 - val_acc: 0.7584
Epoch 6/100
200/200 [==============================] - 17s 84ms/step - loss: 0.9561 - acc: 0.7627 - val_loss: 1.2511 - val_acc: 0.7594
Epoch 7/100
200/200 [==============================] - 17s 84ms/step - loss: 0.9611 - acc: 0.7644 - val_loss: 1.0342 - val_acc: 0.7564
Epoch 8/100
200/200 [==============================] - 

In [ ]:
model.save_weights('/content/drive/My Drive/Colab Notebooks/weights.h5')

### Inference Setup

In [ ]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [ ]:
#Finally, generate the output by defining the following function and later calling it.

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

### Evaluation on Train Dataset

In [ ]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('\033[1m'+ 'Input Russian sentence:' +'\033[0m', X_train[k:k+1].values[0])
print('\033[1m'+ 'Actual English translation:' +'\033[0m', y_train[k:k+1].values[0][6:-4])
print('\033[1m'+ 'Predicted English translation:' +'\033[0m', decoded_sentence[:-4])
nubia.score( y_train[k:k+1].values[0][6:-4], decoded_sentence[:-4], verbose=True)

Input Russian sentence: том неискренний
Actual English translation:  tom is deceitful 
Predicted English translation:  it was tom 
Semantic relation: 0.7205052375793457/5.0
Percent chance of contradiction: 26.138517260551453%
Percent chance of irrelevancy or new information: 63.5387659072876%
Percent chance of logical agreement: 10.322712361812592%

NUBIA score: 0.46372482305638885/1.0


0.46372482305638885

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('\033[1m'+ 'Input Russian sentence:' +'\033[0m', X_train[k:k+1].values[0])
print('\033[1m'+ 'Actual English translation:' +'\033[0m', y_train[k:k+1].values[0][6:-4])
print('\033[1m'+ 'Predicted English translation:' +'\033[0m', decoded_sentence[:-4])
nubia.score( y_train[k:k+1].values[0][6:-4], decoded_sentence[:-4], verbose=True)

Input Russian sentence: сейчас половина четвёртого
Actual English translation:  its 
Predicted English translation:  its 
Semantic relation: 0.9941844940185547/5.0
Percent chance of contradiction: 12.43942603468895%
Percent chance of irrelevancy or new information: 17.079484462738037%
Percent chance of logical agreement: 70.48108577728271%

NUBIA score: 1.0/1.0


1.0

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('\033[1m'+ 'Input Russian sentence:' +'\033[0m', X_train[k:k+1].values[0])
print('\033[1m'+ 'Actual English translation:' +'\033[0m', y_train[k:k+1].values[0][6:-4])
print('\033[1m'+ 'Predicted English translation:' +'\033[0m', decoded_sentence[:-4])
nubia.score( y_train[k:k+1].values[0][6:-4], decoded_sentence[:-4], verbose=True)

Input Russian sentence: я не рад
Actual English translation:  im not happy 
Predicted English translation:  im not happy 
Semantic relation: 5.0/5.0
Percent chance of contradiction: 0.13122111558914185%
Percent chance of irrelevancy or new information: 1.2676193378865719%
Percent chance of logical agreement: 98.60115647315979%

NUBIA score: 1.0/1.0


1.0

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('\033[1m'+ 'Input Russian sentence:' +'\033[0m', X_train[k:k+1].values[0])
print('\033[1m'+ 'Actual English translation:' +'\033[0m', y_train[k:k+1].values[0][6:-4])
print('\033[1m'+ 'Predicted English translation:' +'\033[0m', decoded_sentence[:-4])
nubia.score( y_train[k:k+1].values[0][6:-4], decoded_sentence[:-4], verbose=True)

Input Russian sentence: том утвердительно кивнул
Actual English translation:  tom nodded ok 
Predicted English translation:  you need to rest 
Semantic relation: 1.2755951881408691/5.0
Percent chance of contradiction: 0.6392671260982752%
Percent chance of irrelevancy or new information: 99.14755821228027%
Percent chance of logical agreement: 0.21317268256098032%

NUBIA score: 0.636071633401742/1.0


0.636071633401742

### Evaluation on Validation Dataset

In [ ]:
val_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('\033[1m'+ 'Input Russian sentence:' +'\033[0m', X_test[k:k+1].values[0])
print('\033[1m'+ 'Actual English translation:' +'\033[0m', y_test[k:k+1].values[0][6:-4])
print('\033[1m'+ 'Predicted English translation:' +'\033[0m', decoded_sentence[:-4])
nubia.score( y_test[k:k+1].values[0][6:-4], decoded_sentence[:-4], verbose=True)

Input Russian sentence: том понял
Actual English translation:  tom understood 
Predicted English translation:  tom got me 
Semantic relation: 0.8201012015342712/5.0
Percent chance of contradiction: 0.5283449776470661%
Percent chance of irrelevancy or new information: 18.161186575889587%
Percent chance of logical agreement: 81.31047487258911%

NUBIA score: 0.6968490218583063/1.0


0.6968490218583063

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('\033[1m'+ 'Input Russian sentence:' +'\033[0m', X_test[k:k+1].values[0])
print('\033[1m'+ 'Actual English translation:' +'\033[0m', y_test[k:k+1].values[0][6:-4])
print('\033[1m'+ 'Predicted English translation:' +'\033[0m', decoded_sentence[:-4])
nubia.score( y_test[k:k+1].values[0][6:-4], decoded_sentence[:-4], verbose=True)

Input Russian sentence: что ты сказал
Actual English translation:  what did you say 
Predicted English translation:  what did you say 
Semantic relation: 4.840568542480469/5.0
Percent chance of contradiction: 1.623729057610035%
Percent chance of irrelevancy or new information: 1.166675891727209%
Percent chance of logical agreement: 97.20959663391113%

NUBIA score: 1.0/1.0


1.0

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('\033[1m'+ 'Input Russian sentence:' +'\033[0m', X_test[k:k+1].values[0])
print('\033[1m'+ 'Actual English translation:' +'\033[0m', y_test[k:k+1].values[0][6:-4])
print('\033[1m'+ 'Predicted English translation:' +'\033[0m', decoded_sentence[:-4])
nubia.score( y_test[k:k+1].values[0][6:-4], decoded_sentence[:-4], verbose=True)

Input Russian sentence: том ошибался
Actual English translation:  tom was wrong 
Predicted English translation:  tom was wrong 
Semantic relation: 3.431398868560791/5.0
Percent chance of contradiction: 0.1719208201393485%
Percent chance of irrelevancy or new information: 1.0978682897984982%
Percent chance of logical agreement: 98.73020648956299%

NUBIA score: 1.0/1.0


1.0

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('\033[1m'+ 'Input Russian sentence:' +'\033[0m', X_test[k:k+1].values[0])
print('\033[1m'+ 'Actual English translation:' +'\033[0m', y_test[k:k+1].values[0][6:-4])
print('\033[1m'+ 'Predicted English translation:' +'\033[0m', decoded_sentence[:-4])
nubia.score( y_test[k:k+1].values[0][6:-4], decoded_sentence[:-4], verbose=True)

Input Russian sentence: том улыбается
Actual English translation:  tom is smiling 
Predicted English translation:  is tom that 
Semantic relation: 0.523308515548706/5.0
Percent chance of contradiction: 13.652493059635162%
Percent chance of irrelevancy or new information: 63.55921030044556%
Percent chance of logical agreement: 22.788292169570923%

NUBIA score: 0.6446189319106739/1.0


0.6446189319106739

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('\033[1m'+ 'Input Russian sentence:' +'\033[0m', X_test[k:k+1].values[0])
print('\033[1m'+ 'Actual English translation:' +'\033[0m', y_test[k:k+1].values[0][6:-4])
print('\033[1m'+ 'Predicted English translation:' +'\033[0m', decoded_sentence[:-4])
nubia.score( y_test[k:k+1].values[0][6:-4], decoded_sentence[:-4], verbose=True)

Input Russian sentence: у него не было денег
Actual English translation:  he had no money 
Predicted English translation:  he had no money 
Semantic relation: 4.994417190551758/5.0
Percent chance of contradiction: 0.2987137297168374%
Percent chance of irrelevancy or new information: 0.9756773710250854%
Percent chance of logical agreement: 98.72560501098633%

NUBIA score: 1.0/1.0


1.0

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('\033[1m'+ 'Input Russian sentence:' +'\033[0m', X_test[k:k+1].values[0])
print('\033[1m'+ 'Actual English translation:' +'\033[0m', y_test[k:k+1].values[0][6:-4])
print('\033[1m'+ 'Predicted English translation:' +'\033[0m', decoded_sentence[:-4])
nubia.score( y_test[k:k+1].values[0][6:-4], decoded_sentence[:-4], verbose=True)

Input Russian sentence: здесь я чувствую себя в безопасности
Actual English translation:  i feel safe here 
Predicted English translation:  i feel safe here 
Semantic relation: 5.0/5.0
Percent chance of contradiction: 0.2057696459814906%
Percent chance of irrelevancy or new information: 0.8651942014694214%
Percent chance of logical agreement: 98.92902970314026%

NUBIA score: 1.0/1.0


1.0

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('\033[1m'+ 'Input Russian sentence:' +'\033[0m', X_test[k:k+1].values[0])
print('\033[1m'+ 'Actual English translation:' +'\033[0m', y_test[k:k+1].values[0][6:-4])
print('\033[1m'+ 'Predicted English translation:' +'\033[0m', decoded_sentence[:-4])
nubia.score( y_test[k:k+1].values[0][6:-4], decoded_sentence[:-4], verbose=True)

Input Russian sentence: том обычный
Actual English translation:  tom is average 
Predicted English translation:  tom is the it 
Semantic relation: 0.39058321714401245/5.0
Percent chance of contradiction: 11.835534870624542%
Percent chance of irrelevancy or new information: 35.64484417438507%
Percent chance of logical agreement: 52.519625425338745%

NUBIA score: 0.7691212112414394/1.0


0.7691212112414394